In [1]:
import numpy as np

In [2]:
# Define job class
class Job:
    def __init__(self, mach_req, proc_time, number, rand=0.1):
        self.mach_req = mach_req  # Machines required for the job
        self.proc_time = np.array(proc_time)  # Processing times for the job on each machine
        rand_val = np.ceil(self.proc_time*np.random.random(size=self.proc_time.size)*rand)  # Random values for the job on each machine
        self.proc_time_random = self.proc_time + rand_val  # Randomized processing times for the job on each machine
        self.stage = [False] * len(mach_req)  # Stages of the job completed on machines
        self.counter = 0  # Index to track the current machine for the job
        self.mach_needed = self.mach_req[0]  # Current machine needed for the job
        self.number = number  # Job number
        self.free_time = 0  # Time when the job becomes available for scheduling
        self.job_done = False  # Flag to indicate if the job is completed

    def __lt__(self, other):
        # Comparison method to determine if this job has a shorter processing time
        return self.proc_time[self.counter] < other.proc_time[other.counter]
    
    def __add__(self, other):
        # Add the job's processing time to another object's free time
        return self.proc_time[self.counter] + other.free_time
    
    def __le__(self, other):
        # Comparison method to determine if this job can be scheduled on a machine
        return self.free_time <= other.free_time
    
    def __bool__(self):
        # Boolean representation of a job indicating if it's completed
        return self.job_done
    

# Define machine class
class Machine:
    def __init__(self, number):
        self.free = True  # Flag to indicate if the machine is free
        self.free_time = 0  # Time when the machine becomes available for scheduling
        self.number = number  # Machine number

    def __lt__(self, other):
        # Comparison method to determine if this machine has a shorter free time
        return self.free_time < other.free_time
    
    def __le__(self, other):
        # Comparison method to determine if this machine has an earlier or equal free time
        return self.free_time <= other.free_time



In [3]:
def shortest_job(jobs,mach):
    # Set the number of the machine free
    mach_num = mach.number

    # Initialize arrays for available jobs and their processing times
    job_mach_free = []

    for job in jobs:
        # Get processing times for jobs that require the current machine
        # Considerar como cambia el algoritmo si se pone mach <= job
        if not job.job_done and job.mach_req[job.counter] == mach_num and job <= mach:
            job_mach_free.append(job)
        
    if job_mach_free == []:
        mach.free_time += 1
        return
    
    # Find the job with the shortest processing time
    job_next = min(job_mach_free)
    if job_next.free_time != mach.free_time:
        job_next.free_time = mach.free_time
    
    processing_time = job_next.proc_time_random[job_next.counter]

    # Mark the job as done for this stage
    job_next.stage[job_next.counter] = True
    job_next.counter += 1
    if all(job_next.stage):
        job_next.job_done = True

    return job_next, processing_time


def aleatorio(jobs, machines):
    # Initialize answer list
    # name as follows dict(Task=, Start=, End=, Resource=)
    ans = []

    # Track the order in which jobs pass through each machine
    machine_order = [[] for _ in range(len(machines))]

    while not all(jobs):
        # Find the machine with the earliest free time
        mach = min(machines)

        a = shortest_job(jobs, mach)
        if a != None:
            job_next, processing_time = a[0], a[1]
        else:
            continue

        # Add the job's info to the answer list
        ans.append({
            'Task': 'Job ' + str(job_next.number),
            'Start': mach.free_time,
            'End': mach.free_time + processing_time,
            'Resource': 'M. ' + str(mach.number)
        })

        # Update the machine's free time
        job_next.free_time += processing_time
        mach.free_time += processing_time
    
        # Track the order in which this job passed through the machine
        machine_order[mach.number - 1].append(job_next.number)
    
    return ans, machine_order

In [9]:
def calculate_makespan(schedule, processing_time):
    n = len(processing_time)
    m = len(processing_time[0])

    # Initialize completion times for each machine
    machine_completion_times = [0] * m

    # Initialize completion times for each job
    job_completion_times = [0] * n

    for j, machine_order in enumerate(schedule):
        for job_number in machine_order:
            machine_number = j  # Machine number corresponds to the index in the list
            job_number -= 1  # Adjust job number to match list indexing (0-based)

            # Calculate the start time of the job on the machine
            start_time = max(job_completion_times[job_number], machine_completion_times[machine_number])

            # Update the completion times for the job and the machine
            job_completion_times[job_number] = start_time + processing_time[job_number][machine_number]
            machine_completion_times[machine_number] = start_time + processing_time[job_number][machine_number]

    # The makespan is the maximum completion time of any job
    makespan = max(job_completion_times)
    
    return makespan

# Example usage:
# schedule is a list of lists containing the order of jobs passing through each machine
# processing_time is a list of lists containing processing times for each job on each machine
calculate_makespan(machine_order, processing_time)


89

In [4]:
#with open(directory_path + "/" + file, "r" ) as f:
with open("AAA_test", "r" ) as f:
    size = f.readline().split()
    # n = trabajos, m = maquinas
    n, m = int(size[0]), int(size[1])
    data = list(f)
    for i,line in enumerate(data):
      data[i] = [int(x) for x in line.strip().split() if x is not None and x != '']

    processing_time = data[0:n]
    machines_required = data[n:2*n]

# Create job objects
jobs = [Job(np.array(machines_required)[i, :], np.array(processing_time)[i, :], i + 1, 0) for i in range(n)]

#Sorted array in which the first value is always the first machine that 
machines = [Machine(i+1) for i in range(m)]

ans, machine_order = aleatorio(jobs, machines)


In [5]:
machine_order

[[2, 4, 3, 1], [4, 3, 2, 1], [3, 1, 2, 4]]